## Alocação de unidades de atendimento móvel do CBMERJ de Campos dos Goytacazes - RJ

In [6]:
# Library
import pathlib
import os
import sys

import numpy as np
import pandas as pd
from pulp import *
from utils.process import stander_text
# my packges
sys.path.insert(0, str(pathlib.Path().absolute().parent))

import process.allocation as allocation
from process.geocode import *

import plotly.express as px
import plotly.io as pio


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\fabio\Documents\PMedian\env\Lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
  File "C:\Users\fabio\AppData\Local\Temp\ipykernel_704\3053745432.py", line 6, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\fabio\Documents\PMedian\env\Lib\site-packages\pygments\styles\__init__.py", line 90, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\fabio\Documents\PMedian\env\Lib\site-packages\IPython\core\interactiveshell.py", line 2142, in showtraceback
  File "c:\Users\fabio\Documents\PMedian\env\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
  File "c:\Users\fabio\Documents\PMedi

In [2]:
pio.templates.default = "seaborn"

**Importar dados**

In [3]:
df_matrix= pd.read_excel('../data/matriz_tempo.xlsx', index_col=0)

In [4]:
df_matrix

,5 GBM,DBM 1/5 GUARUS,HOSP. FERREIRA M,HOSP. G. GUARUS,HOSP. SÃO JOSÉ,HOSP. TRAVESSÃO,PS GUARUS,SANDU,UBS Sartunino Braga,UBS TOCOS,UPA,UPH F. SÃO THOMÉ,UPH MORRO DO COCO,UPH ST EDUARDO
Clientes,,,,,,,,,,,,,,
Alphaville,1132,1997,1449,1345,527,2579,1450,972,1610,1348,1706,3454,4157,5609
Baixa Grande,3004,3869,3321,3217,1923,4451,3322,2844,655,2062,3578,1189,6029,7481
Boa Vista,3536,4401,3852,3749,2454,4982,3854,3375,1186,2432,4109,673,6561,8012
Caboio,2808,3673,3124,3021,1726,4254,3126,2647,1025,1404,3381,1925,5833,7284
Caju,508,768,150,591,1712,1331,368,811,2794,2422,412,4638,2909,4361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vila Industrial,982,223,909,800,2304,1050,826,1323,3386,3124,521,5231,2629,4080
Vila MANHÃES,1077,1942,1394,1290,974,2524,1455,999,1524,1571,1650,3369,4102,5554
Vila Nova de Campos,2751,2197,2625,2570,4073,1425,2596,3093,5155,4893,2238,7000,1052,2580


# Modelo matemático

In [5]:
# DEFININDO VARIÁVEIS
location = df_matrix.columns.values
clients = df_matrix.index.values 
w = [1 for w in clients]
m = df_matrix.shape[0] # Nº de clientes
n = df_matrix.shape[1] # Nº de potenciais facilidades
p = 9 # Nº de facilidades   


### Cenários
**Cenário base**

In [6]:
# Cenario base
bases_atuais = ['5 GBM', 'DBM 1/5 GUARUS', 'HOSP. FERREIRA M', 'HOSP. G. GUARUS', 'HOSP. SÃO JOSÉ', 'HOSP. TRAVESSÃO', 'PS GUARUS', 'SANDU', 'UBS Sartunino Braga']
scenario_base = df_matrix[bases_atuais].apply(min, axis=1).copy()
df_matrix[bases_atuais].apply(min, axis=1).describe()

count     165.000000
mean      732.703030
std       701.194166
min        43.000000
25%       327.000000
50%       499.000000
75%       817.000000
max      3687.000000
dtype: float64

**Cenário ponderado**

In [7]:
# Carregar arquivo de pesos
df_weight = pd.read_csv('../data/pesos.csv', delimiter=';')
# Agregar para eliminar valores duplicados
df_weight = df_weight.groupby('Clientes').sum().reset_index()

In [8]:
df_bairros_ponderado = df_matrix.copy()
df_bairros_ponderado = df_bairros_ponderado.reset_index()[['Clientes']]
df_bairros_ponderado = df_bairros_ponderado.merge(df_weight, left_on='Clientes', right_on='Clientes', how='left')
df_bairros_ponderado.head()

,Clientes,Bairros,2017,2018,2019,2020,Total
0,Alphaville,Alphaville,28.0,9.0,12.0,0.0,49
1,Baixa Grande,Baixa Grande,0.0,0.0,6.0,0.0,6
2,Boa Vista,Boa Vista,18.0,62.0,124.0,0.0,204
3,Caboio,Caboio,0.0,4.0,0.0,0.0,4
4,Caju,Caju,43.0,87.0,88.0,0.0,218


In [9]:
scenarios = {
    'Não ponderado':[1 for x in range(m)],
    'Ponderado':df_bairros_ponderado.Total.to_list()
}

**Rodar modelo**

In [10]:
# Lista contendo os resultado dos cenários
results = []
for scenario, weigts in scenarios.items():
    p_median = allocation.Allocation(m, n, p, df_matrix, weigts)
    p_median.solver()
    res = p_median.get_result()
    results.append({scenario:res})

1
Optimal objective value: 111204.0
Facility 1 is located. DBM 1/5 GUARUS
- Customer 12 is served. Codim
- Customer 22 is served. Estrada de Brejo Grande
- Customer 49 is served. Novo Eldorado - Parque CEASA
- Customer 51 is served. Parque Aeroporto
- Customer 74 is served. Parque Eldorado
- Customer 102 is served. Parque Santa Rosa
- Customer 105 is served. Parque Santos Dumont
- Customer 110 is served. Parque Sao Silvestre
- Customer 126 is served. Parque palmares
- Customer 148 is served. São Sebastião
- Customer 151 is served. Terra Prometida
- Customer 160 is served. Vila Industrial
Facility 2 is located. HOSP. FERREIRA M
- Customer 4 is served. Caju
- Customer 11 is served. Chácara dos Sonhos - Morro Grande
- Customer 18 is served. Dores De Macabu
- Customer 29 is served. Guriri - Dores de Macabu
- Customer 32 is served. Ibitioca
- Customer 36 is served. Jardim Maria de Queiroz
- Customer 38 is served. Lagoa De Cima
- Customer 42 is served. Mato Escuro
- Customer 45 is served. Mo

In [11]:
class GetResultPmedian:
    def __init__(self, results):
        self.results = results    
    
    def struct_results(self: list):
        self.scenario_results = {}
        self.open_facilities = {}
        self.times = {}
        stats = []
        for res in self.results:
            key, value = zip(*res.items())
            self.open_facilities.update({key[0]:value[0][0]})
            self.scenario_results.update({
                key[0]:['\n'.join(value[0][0])]
            }) 
            stats.append(
                pd.Series(value[0][1]).describe().reset_index().set_index('index').T
            )
            self.times.update({key:value[0][1]})
        df_stats = pd.concat(stats, ignore_index=True)
        df = pd.DataFrame(self.scenario_results).melt(var_name='Cenários', value_name='Bases')
        return df.join(df_stats)
    
    def get_facilities_open(self):
        structured_data_from_open_facilities = {key:[] for key in ['bases',*self.open_facilities.keys()]}
        facilities = self._get_all_facilities()
        structured_data_from_open_facilities['bases'].extend(facilities)
        
        for base in facilities:
            for scenario, open_bases in self.open_facilities.items():
                if base in open_bases:
                    structured_data_from_open_facilities[scenario].append(1)  
                else:
                    structured_data_from_open_facilities[scenario].append(0) 
                
        return pd.DataFrame(structured_data_from_open_facilities)
    
    def get_times(self):
        return pd.DataFrame(self.times)
    
    def _get_all_facilities(self):
        _,_,matrix = zip(*self.results[0].values())
        return matrix[0].columns

In [12]:
res_pmedian = GetResultPmedian(results)

struct_results = res_pmedian.struct_results()
struct_results.to_excel('cenarios.xlsx')    
struct_results

,Cenários,Bases,count,mean,std,min,25%,50%,75%,max
0,Não ponderado,DBM 1/5 GUARUS\nHOSP. FERREIRA M\nHOSP. SÃO JO...,165.0,673.963636,648.410626,0.0,315.0,450.0,712.0,3687.0
1,Ponderado,5 GBM\nDBM 1/5 GUARUS\nHOSP. FERREIRA M\nHOSP....,165.0,781.290909,821.582063,43.0,307.0,453.0,880.0,4120.0


In [13]:
facilities_open = res_pmedian.get_facilities_open()
facilities_open.to_excel('bases_abertas.xlsx')  
facilities_open   

,bases,Não ponderado,Ponderado
0,5 GBM,0,1
1,DBM 1/5 GUARUS,1,1
2,HOSP. FERREIRA M,1,1
3,HOSP. G. GUARUS,0,0
4,HOSP. SÃO JOSÉ,1,1
5,HOSP. TRAVESSÃO,1,1
6,PS GUARUS,0,1
7,SANDU,1,1
8,UBS Sartunino Braga,1,1
9,UBS TOCOS,1,0


In [14]:
fig = px.box(
    res_pmedian.get_times().melt(var_name='Cenario'),
    x='value',
    facet_row='Cenario',
    title='Tempo de deslocamento - segundos',
    labels={'value':'Tempo de deslocamento (s)'},
    height=600,
    width=800,
)
fig.show()

# Alterar os nomes dos subgráficos
# fig.for_each_annotation(lambda name: name.update(text=name.text.replace("variable=", "")))
fig.write_image("boxplot.svg")
 


In [5]:
%pip install -r 'requirements.txt'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: "'requirements.txt'"

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
